In [ ]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
batch_size = 64
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
device = torch.device("cpu")#torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
train_dataset = datasets.MNIST(root='./dataset/mnist/', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_dataset = datasets.MNIST(root='./dataset/mnist/', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=1,out_channels=10,kernel_size=5)
        self.conv2 = torch.nn.Conv2d(in_channels=10,out_channels=20,kernel_size=5)
        self.conv3 = torch.nn.Conv2d(in_channels=20,out_channels=20,kernel_size=3)
        self.maxpool = torch.nn.MaxPool2d(2) 
        self.linear = torch.nn.Linear(80, 10)
 
    def forward(self, x):
        batch_size = x.size(0)
        x = self.maxpool(self.conv1(x)) # 10*12*12
        x = self.maxpool(self.conv2(x)) # 20*4*4
        x = self.conv3(x) # 20*2*2
        x = x.view(batch_size,-1)
        x = self.linear(x)
        return x

In [ ]:
model = Net()
model.to(device=device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [ ]:
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        # 获得一个批次的数据和标签
        inputs, target = data
        inputs, target = inputs.to(device=device), target.to(device=device)
        optimizer.zero_grad()
        # 获得模型预测结果(64, 10)
        outputs = model(inputs)
        # 交叉熵代价函数outputs(64,10),target（64）
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
 
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch+1, batch_idx+1, running_loss/300))
            running_loss = 0.0
 
 
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device=device), labels.to(device=device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1) # dim = 1 列是第0个维度，行是第1个维度
            total += labels.size(0)
            correct += (predicted == labels).sum().item() # 张量之间的比较运算
    return correct/total

In [ ]:
if __name__ == '__main__':
    accuracy_max = 0
    accuracy = 0
    accuracys = []
    for epoch in range(10):
        train(epoch)
        accuracy = test()
        if(accuracy_max < accuracy):
            accuracy_max = accuracy
        accuracys.append(accuracy)
    print("accuracy (%d,%d)"%(accuracy* 100,accuracy_max * 100))
    plt.plot(accuracys)